## Exercises

### Exercise 1

A) Write a program implementing a linear congruential generator (LCG). Be sure that the program works correctly using only integer representation.

**Linear Congruential Generator (LCG)**

An LCG has the form:

$$
X_{n+1} = (a \cdot X_n + c) \mod m
$$

Where:

- \( a \): multiplier  
- \( c \): shift 
- \( m \): modulus  
- \( X_0 \): initial value (seed)


In [ ]:
# LCG parameters
# Linear Congruential Generator (LCG) implementation
a = 5
c = 1
m = 16
seed = 237

def lcg(n, seed=seed):
    x = seed
    numbers = []
    for _ in range(n):
        x = (a * x + c) % m
        numbers.append(x / m)  # Normaliser til [0,1)
    return numbers

# Generate 10000 random numbers
numbers = lcg(10000)

# Parameters for [0,1) interval
num_bins = 10
min_val = 0.0
max_val = 1.0
bin_width = (max_val - min_val) / num_bins

# Initialize bins
hist_counts = [0] * num_bins

# Count occurrences in each bin
for val in numbers:
    bin_index = int((val - min_val) / bin_width)
    if bin_index >= num_bins:  # Clamp to last bin if val == 1.0 (rare but safe)
        bin_index = num_bins - 1
    hist_counts[bin_index] += 1

# Determine max count for scaling
max_count = max(hist_counts)

# Build text histogram
histogram_text = ""
for i in range(num_bins):
    start = min_val + i * bin_width
    end = start + bin_width
    bar = '█' * int((hist_counts[i] / max_count) * 40)
    label = f"[{start:.1f}, {end:.1f})"
    histogram_text += f"{label.ljust(14)} | {bar} ({hist_counts[i]})\n"

print(histogram_text)



[0.0, 0.1)     | ████████████████████████████████████████ (1250)
[0.1, 0.2)     | ████████████████████████████████████████ (1250)
[0.2, 0.3)     | ████████████████████ (625)
[0.3, 0.4)     | ████████████████████████████████████████ (1250)
[0.4, 0.5)     | ████████████████████ (625)
[0.5, 0.6)     | ████████████████████████████████████████ (1250)
[0.6, 0.7)     | ████████████████████████████████████████ (1250)
[0.7, 0.8)     | ████████████████████ (625)
[0.8, 0.9)     | ████████████████████████████████████████ (1250)
[0.9, 1.0)     | ████████████████████ (625)



#### B

Evaluate the quality of the generator by graphical descriptive statistics (histogrammes, scatter plots) and statistical tests- χ2,Kolmogorov-Smirnov, run-tests preferably but not necessarily all 3, and correlation test forsome h-values.

In [13]:
# Show 20 x_i
print(" i     X_i     ")
print("--------------")
for i in range(20):
    xi = round(numbers[i], 4)
    print(f"{i:<5} {xi:<10} ")


 i     X_i     
--------------
0     0.125      
1     0.6875     
2     0.5        
3     0.5625     
4     0.875      
5     0.4375     
6     0.25       
7     0.3125     
8     0.625      
9     0.1875     
10    0.0        
11    0.0625     
12    0.375      
13    0.9375     
14    0.75       
15    0.8125     
16    0.125      
17    0.6875     
18    0.5        
19    0.5625     


#### Chi_squared test

In [14]:
expected = len(numbers) // num_bins
chi_squared = 0
for observed in hist_counts:
    chi_squared += ((observed - expected) ** 2) / expected

print(f"\nChi-squared statistic: {chi_squared:.2f}")



Chi-squared statistic: 937.50


In [ ]:
# SLETTES???????



run_count = 1
prev_trend = None
short_seq = numbers[:100]  # take first 100 values
for i in range(1, len(short_seq)):
    if short_seq[i] > short_seq[i - 1]:
        trend = "up"
    elif short_seq[i] < short_seq[i - 1]:
        trend = "down"
    else:
        trend = "same"

    if trend != prev_trend and trend != "same":
        run_count += 1
        prev_trend = trend

print(f"\nEstimated number of runs (first 100 values): {run_count}")



Estimated number of runs (first 100 values): 64


#### Correlation test (Rikke, ikke den fra slides)

In [16]:
n = len(numbers) - 1
x = numbers[:-1]
y = numbers[1:]
mean_x = sum(x) / n
mean_y = sum(y) / n

numerator = sum((x[i] - mean_x) * (y[i] - mean_y) for i in range(n))
denominator_x = sum((x[i] - mean_x) ** 2 for i in range(n))
denominator_y = sum((y[i] - mean_y) ** 2 for i in range(n))
correlation = numerator / (denominator_x * denominator_y) ** 0.5

print(f"\nCorrelation between X_i and X_(i+1): {correlation:.4f}")



Correlation between X_i and X_(i+1): 0.2708


#### Correlation test (Lotte, den fra slides)

In [35]:
def lag_correlation(U, h=1):
    n = len(U)
    return sum(U[i] * U[i + h] for i in range(n - h)) / (n - h)

ch = lag_correlation(numbers, h=1)
print(f"Estimated c₁ = {ch:.5f}")


Estimated c₁ = 0.24668


#### Kolmogorov Smirnov test

In [17]:
def ks_test_uniform(data):
    n = len(data)
    sorted_data = sorted(data)

    # Compute empirical CDF and compare with theoretical CDF
    D_plus = max((i + 1) / n - val for i, val in enumerate(sorted_data))
    D_minus = max(val - i / n for i, val in enumerate(sorted_data))

    Dn = max(D_plus, D_minus)
    return Dn

def ks_critical_value(alpha=0.05, n=10000):
    # Critical value constant for alpha = 0.05
    c_alpha = 1.36
    return c_alpha / (n ** 0.5)


In [ ]:
# ks test
Dn = ks_test_uniform(numbers)
D_crit = ks_critical_value(0.05, len(numbers))

print(f"KS Statistic: {Dn:.5f}")
print(f"Critical value (alpha=0.05): {D_crit:.5f}")

if Dn > D_crit:
    print("Reject null hypothesis: sample is not uniform.")
else:
    print("Fail to reject null hypothesis: sample appears uniform.")

KS Statistic: 0.06250
Critical value (alpha=0.05): 0.01360
Reject null hypothesis: sample is not uniform.


#### Run test I

In [26]:
import numpy as np
from math import sqrt

def run_test_above_below(data):
    median = np.median(data)
    
    # Step 1: Convert to binary sequence: 1 if above median, 0 if below
    signs = [1 if x > median else 0 for x in data]
    
    # Step 2: Count runs (change between 0 and 1)
    runs = 1  # first element starts the first run
    for i in range(1, len(signs)):
        if signs[i] != signs[i - 1]:
            runs += 1

    n1 = signs.count(1)  # number above median
    n2 = signs.count(0)  # number below median

    if n1 == 0 or n2 == 0:
        return runs, None, None, "Not enough variation to perform test"

    # Step 3: Calculate expected value and variance under H0
    expected_runs = 2 * n1 * n2 / (n1 + n2) + 1
    variance_runs = (
        2 * n1 * n2 * (2 * n1 * n2 - n1 - n2)
        / ((n1 + n2) ** 2 * (n1 + n2 - 1))
    )

    # Step 4: Compute Z-score
    z = (runs - expected_runs) / sqrt(variance_runs)

    return runs, expected_runs, z, None


In [27]:
# Run the test
runs, expected, z, warning = run_test_above_below(numbers)

print(f"Observed number of runs: {runs}")
print(f"Expected number of runs: {expected:.2f}")
print(f"Z-score: {z:.3f}" if z is not None else warning)

# Interpret result
if z is not None:
    if abs(z) > 1.96:
        print("Reject null hypothesis: evidence of non-randomness.")
    else:
        print("Fail to reject null hypothesis: runs appear random.")


Observed number of runs: 5043
Expected number of runs: 5001.00
Z-score: 0.840
Fail to reject null hypothesis: runs appear random.


#### Run test II

In [31]:
import numpy as np

def run_test_knuth(data):
    n = len(data)

    # Step 1: Count run lengths
    run_lengths = []
    current_run = 1
    for i in range(1, n):
        if data[i] > data[i - 1]:
            current_run += 1
        else:
            run_lengths.append(current_run)
            current_run = 1
    run_lengths.append(current_run)  # include final run

    # Step 2: Build vector R (run count for length 1-5, and 6+)
    R = np.zeros(6)
    for length in run_lengths:
        if length >= 6:
            R[5] += 1
        else:
            R[length - 1] += 1

    # Step 3: Define A matrix (6x6) and B vector (6x1)
    A = np.array([
        [4529.4, 9044.9, 13568, 18091, 22615, 27892],
        [9044.9, 18097, 27139, 36187, 45234, 55789],
        [13568, 27139, 40721, 54281, 67852, 83685],
        [18091, 36187, 54281, 72414, 90470, 111580],
        [22615, 45234, 67852, 90470, 113262, 139476],
        [27892, 55789, 83685, 111580, 139476, 172860]
    ])

    B = np.array([1/6, 5/24, 11/120, 19/720, 29/5040, 1/840])

    # Step 4: Compute test statistic Z
    diff = R - n * B
    Z = (1 / (n - 6)) * diff @ A @ diff.T
    return Z, R


In [32]:
# Perform the run test
Z, R = run_test_knuth(numbers)

print(f"Z-statistic: {Z:.3f}")
print("Run counts (lengths 1–5, 6+):", R.astype(int))

# Interpret using chi-square distribution with 6 degrees of freedom
from scipy.stats import chi2
p_value = 1 - chi2.cdf(Z, df=6)
print(f"p-value: {p_value:.4f}")


Z-statistic: 3.279
Run counts (lengths 1–5, 6+): [1652 2099  922  256   52   16]
p-value: 0.7731


#### Run test III

In [33]:
def up_down_test(data):
    if len(data) < 2:
        return None, None

    # Step 1: Convert data to up/down sequence: 1 for up, -1 for down, 0 for equal
    directions = []
    for i in range(1, len(data)):
        if data[i] > data[i - 1]:
            directions.append(1)
        elif data[i] < data[i - 1]:
            directions.append(-1)

    # Step 2: Count runs in the direction sequence
    if not directions:
        return 0, 0

    runs = 1
    for i in range(1, len(directions)):
        if directions[i] != directions[i - 1]:
            runs += 1

    # Step 3: Compute expected value and variance
    n = len(directions) + 1  # original sample length
    expected = (2 * n - 1) / 3
    variance = (16 * n - 29) / 90

    # Step 4: Compute Z-score
    z = (runs - expected) / sqrt(variance)

    return z, runs


In [34]:
z, total_runs = up_down_test(numbers)
print(f"Number of up/down runs: {total_runs}")
print(f"Z-score: {z:.4f}")

# Optional interpretation
if abs(z) > 1.96:
    print("Reject null hypothesis: sequence shows non-random behavior.")
else:
    print("Fail to reject null hypothesis: no evidence against randomness.")


Number of up/down runs: 6690
Z-score: 0.5614
Fail to reject null hypothesis: no evidence against randomness.


### Øvelse 1.2

In [37]:
import random

sim_numbers = [random.random() for _ in range(10000)]

In [41]:
# Parameters for [0,1) interval
num_bins = 10
min_val = 0.0
max_val = 1.0
bin_width = (max_val - min_val) / num_bins

# Initialize bins
hist_counts = [0] * num_bins

# Count occurrences in each bin
for val in sim_numbers:
    bin_index = int((val - min_val) / bin_width)
    if bin_index >= num_bins:  # Clamp to last bin if val == 1.0 (rare but safe)
        bin_index = num_bins - 1
    hist_counts[bin_index] += 1

# Determine max count for scaling
max_count = max(hist_counts)

# Build text histogram
histogram_text = ""
for i in range(num_bins):
    start = min_val + i * bin_width
    end = start + bin_width
    bar = '█' * int((hist_counts[i] / max_count) * 40)
    label = f"[{start:.1f}, {end:.1f})"
    histogram_text += f"{label.ljust(14)} | {bar} ({hist_counts[i]})\n"

print(histogram_text)

[0.0, 0.1)     | ████████████████████████████████████████ (1044)
[0.1, 0.2)     | ████████████████████████████████████ (962)
[0.2, 0.3)     | ██████████████████████████████████████ (1000)
[0.3, 0.4)     | ██████████████████████████████████████ (1014)
[0.4, 0.5)     | ██████████████████████████████████████ (997)
[0.5, 0.6)     | ████████████████████████████████████ (958)
[0.6, 0.7)     | ███████████████████████████████████████ (1037)
[0.7, 0.8)     | ██████████████████████████████████████ (1013)
[0.8, 0.9)     | ██████████████████████████████████████ (1014)
[0.9, 1.0)     | ████████████████████████████████████ (961)



In [ ]:
# Not nessary to show 20 x_i for simulated numbers, but can be done similarly?!?
# Show 20 x_i 
print(" i     X_i     ")
print("--------------")
for i in range(20):
    xi = round(numbers[i], 4)
    print(f"{i:<5} {xi:<10} ")


#### Chi_squared for sim_numbers

In [42]:
expected = len(sim_numbers) // num_bins
chi_squared = 0
for observed in hist_counts:
    chi_squared += ((observed - expected) ** 2) / expected

print(f"\nChi-squared statistic: {chi_squared:.2f}")


Chi-squared statistic: 8.60


#### KS-test

In [43]:
# Run test
Dn = ks_test_uniform(sim_numbers)
D_crit = ks_critical_value(0.05, len(sim_numbers))

print(f"KS Statistic: {Dn:.5f}")
print(f"Critical value (alpha=0.05): {D_crit:.5f}")

if Dn > D_crit:
    print("Reject null hypothesis: sample is not uniform.")
else:
    print("Fail to reject null hypothesis: sample appears uniform.")

KS Statistic: 0.00602
Critical value (alpha=0.05): 0.01360
Fail to reject null hypothesis: sample appears uniform.


#### Correlation test

In [44]:
ch = lag_correlation(sim_numbers, h=1)
print(f"Estimated c₁ = {ch:.5f}")

Estimated c₁ = 0.24830


In [47]:
# Run test 1

# Run the test
runs, expected, z, warning = run_test_above_below(sim_numbers)

print(f"Observed number of runs: {runs}")
print(f"Expected number of runs: {expected:.2f}")
print(f"Z-score: {z:.3f}" if z is not None else warning)

# Interpret result
if z is not None:
    if abs(z) > 1.96:
        print("Reject null hypothesis: evidence of non-randomness.")
    else:
        print("Fail to reject null hypothesis: runs appear random.")

Observed number of runs: 4983
Expected number of runs: 5001.00
Z-score: -0.360
Fail to reject null hypothesis: runs appear random.


In [48]:
# Run test 2

# Perform the run test
Z, R = run_test_knuth(sim_numbers)

print(f"Z-statistic: {Z:.3f}")
print("Run counts (lengths 1–5, 6+):", R.astype(int))

# Interpret using chi-square distribution with 6 degrees of freedom
from scipy.stats import chi2
p_value = 1 - chi2.cdf(Z, df=6)
print(f"p-value: {p_value:.4f}")

Z-statistic: 9.292
Run counts (lengths 1–5, 6+): [1654 2125  879  288   45   13]
p-value: 0.1578


In [49]:
# Run test 3

z, total_runs = up_down_test(sim_numbers)
print(f"Number of up/down runs: {total_runs}")
print(f"Z-score: {z:.4f}")

# Optional interpretation
if abs(z) > 1.96:
    print("Reject null hypothesis: sequence shows non-random behavior.")
else:
    print("Fail to reject null hypothesis: no evidence against randomness.")

Number of up/down runs: 6700
Z-score: 0.7985
Fail to reject null hypothesis: no evidence against randomness.
